In [18]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from app.utils import CustomImputer, CyclicalFeatureTransformer

In [19]:
# 2. LOAD & PREPARE DATA
df = pd.read_csv("C:/Users/ASUS/PROGRAMMING LANGUAGE/ASAH_Capstone Project Fix/bank-additional-full.csv", sep= ';')
df.rename(columns = {'y':'deposit'}, inplace = True)
y = df['deposit'].map({'yes': 1, 'no': 0})
X = df.drop('deposit', axis=1)

# Fitur Final (10 Fitur Deployment)
final_features = [
    'euribor3m', 'nr.employed', 'age', 'cons.conf.idx', 'campaign',
    'poutcome', 'previous', 'job', 'education', 'month'
]

X_filtered = X[final_features].copy()
X_train, X_test, y_train, y_test = train_test_split(
    X_filtered, y, test_size=0.2, random_state=42, stratify=y
)

In [20]:
# 3. DEFINISI CUSTOM TRANSFORMERS
class CustomImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):
        X_copy = X.copy()
        if 'job' in X_copy.columns and 'age' in X_copy.columns:
            X_copy.loc[(X_copy['age']>60) & (X_copy['job']=='unknown'), 'job'] = 'retired'
        if 'education' in X_copy.columns and 'job' in X_copy.columns:
            X_copy.loc[(X_copy['education']=='unknown') & (X_copy['job']=='management'), 'education'] = 'university.degree'
            X_copy.loc[(X_copy['education']=='unknown') & (X_copy['job']=='services'), 'education'] = 'high.school'
            X_copy.loc[(X_copy['education']=='unknown') & (X_copy['job']=='housemaid'), 'education'] = 'basic.4y'
            X_copy.loc[(X_copy['job']=='unknown') & (X_copy['education']=='professional.course'), 'job'] = 'technician'
            basic_ed = ['basic.4y', 'basic.6y', 'basic.9y']
            X_copy.loc[(X_copy['job'] == 'unknown') & (X_copy['education'].isin(basic_ed)), 'job'] = 'blue-collar'
        return X_copy

class CyclicalFeatureTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):
        X_copy = X.copy()
        new_features = []
        if 'month' in X_copy.columns:
            month_map = {'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
            month_num = X_copy['month'].map(month_map).fillna(0)
            X_copy['month_sin'] = np.sin(2 * np.pi * month_num/12)
            X_copy['month_cos'] = np.cos(2 * np.pi * month_num/12)
            new_features.extend(['month_sin', 'month_cos'])
            X_copy.drop(columns=['month'], inplace=True)
        if new_features: return X_copy
        return X_copy
    def get_feature_names_out(self, input_features=None): return ['month_sin', 'month_cos']

# 4. PREPROCESSOR (SAMA SEPERTI KODE ANDA)
education_order = ['illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'professional.course', 'university.degree', 'unknown']
poutcome_order = ['nonexistent', 'failure', 'success']
sel_num_features = ['euribor3m', 'nr.employed', 'age', 'cons.conf.idx', 'campaign', 'previous']
sel_cat_ordinal  = ['education', 'poutcome']
sel_cat_onehot   = ['job']
sel_cyclic       = ['month']

preprocessor_deploy = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), sel_num_features),
        ('ord', OrdinalEncoder(categories=[education_order, poutcome_order], handle_unknown='use_encoded_value', unknown_value=-1), sel_cat_ordinal),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), sel_cat_onehot),
        ('cyclic', CyclicalFeatureTransformer(), sel_cyclic)
    ], remainder='drop', verbose_feature_names_out=False
)

In [21]:
# 5. DEFINISI MODEL

# Hitung bobot untuk XGBoost (karena tidak punya 'balanced' otomatis)
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
scale_pos_weight_val = neg_count / pos_count

models_deploy = {}

# --- A. Model Klasik ---
models_deploy["Logistic Regression"] = Pipeline(steps=[
    ('imputer', CustomImputer()),
    ('preprocessor', preprocessor_deploy),
    ('classifier', LogisticRegression(random_state=42, solver='liblinear', C=1.0, class_weight='balanced'))
])

models_deploy["Naive Bayes"] = Pipeline(steps=[
    ('imputer', CustomImputer()),
    ('preprocessor', preprocessor_deploy),
    ('classifier', GaussianNB())
])

models_deploy["Random Forest"] = Pipeline(steps=[
    ('imputer', CustomImputer()),
    ('preprocessor', preprocessor_deploy),
    ('classifier', RandomForestClassifier(random_state=42, n_estimators=200, class_weight='balanced'))
])

# --- B. Model Boosting ---
models_deploy["XGBoost"] = Pipeline(steps=[
    ('imputer', CustomImputer()),
    ('preprocessor', preprocessor_deploy),
    ('classifier', XGBClassifier(
        random_state=42,
        scale_pos_weight=scale_pos_weight_val, # Handle Imbalance
        use_label_encoder=False,
        eval_metric='logloss'
    ))
])

models_deploy["LightGBM"] = Pipeline(steps=[
    ('imputer', CustomImputer()),
    ('preprocessor', preprocessor_deploy),
    ('classifier', LGBMClassifier(
        random_state=42,
        class_weight='balanced', # Handle Imbalance
        verbose=-1
    ))
])

models_deploy["CatBoost"] = Pipeline(steps=[
    ('imputer', CustomImputer()),
    ('preprocessor', preprocessor_deploy),
    ('classifier', CatBoostClassifier(
        random_state=42,
        auto_class_weights='Balanced', # Handle Imbalance
        verbose=0
    ))
])

In [22]:
# 6. TRAINING & EVALUASI OTOMATIS

best_auc = 0
best_model_name = ""
best_model_pipeline = None
results_summary = []

for name, model in models_deploy.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)

    # Prediksi
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Metrik
    auc = roc_auc_score(y_test, y_proba)

    print(f"--- Laporan: {name} ---")
    print(classification_report(y_test, y_pred, target_names=['No Deposit', 'Deposit']))
    print(f"AUC Score: {auc:.4f}")

    results_summary.append({'Model': name, 'AUC': auc})

    # Cek Terbaik
    if auc > best_auc:
        best_auc = auc
        best_model_name = name
        best_model_pipeline = model


Training Logistic Regression...
--- Laporan: Logistic Regression ---
              precision    recall  f1-score   support

  No Deposit       0.95      0.76      0.85      7310
     Deposit       0.28      0.72      0.40       928

    accuracy                           0.76      8238
   macro avg       0.62      0.74      0.62      8238
weighted avg       0.88      0.76      0.80      8238

AUC Score: 0.7797

Training Naive Bayes...
--- Laporan: Naive Bayes ---
              precision    recall  f1-score   support

  No Deposit       0.93      0.86      0.89      7310
     Deposit       0.30      0.47      0.36       928

    accuracy                           0.81      8238
   macro avg       0.61      0.66      0.63      8238
weighted avg       0.86      0.81      0.83      8238

AUC Score: 0.7500

Training Random Forest...
--- Laporan: Random Forest ---
              precision    recall  f1-score   support

  No Deposit       0.92      0.95      0.93      7310
     Deposit       

In [23]:
# 7. RINGKASAN HASIL

print(f" MODEL TERBAIK: {best_model_name}")
print(f" AUC Score: {best_auc:.4f}")
print("\Hasil Keseluruhan Model:")
print(pd.DataFrame(results_summary).sort_values(by='AUC', ascending=False))

 MODEL TERBAIK: LightGBM
 AUC Score: 0.8095
\Hasil Keseluruhan Model:
                 Model       AUC
4             LightGBM  0.809540
5             CatBoost  0.801865
3              XGBoost  0.787214
0  Logistic Regression  0.779726
2        Random Forest  0.768094
1          Naive Bayes  0.750037


In [24]:
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier

# KONSTANTA TUNING
N_TRIALS = 50
RANDOM_STATE = 42
SCORING_METRIC = 'roc_auc' # Metrik yang dioptimalkan


def objective(trial):
    """Fungsi yang dioptimalkan oleh Optuna."""

    # 1. Definisi Parameter Space untuk LightGBM (Leaf-Wise Control)
    lgbm_params = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),       # Kunci Kontrol Leaf-Wise
        'max_depth': trial.suggest_int('max_depth', 4, 10),           # Batasan eksplisit
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 200), # Sama dengan min_data_in_leaf
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),    # Mencegah overfitting
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0), # Mencegah overfitting
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0), # L1 regularization
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0), # L2 regularization
        'random_state': RANDOM_STATE,
        'class_weight': 'balanced',
        'n_jobs': -1,
        'verbose': -1
    }

    # 2. Pipeline Model
    model = Pipeline(steps=[
        ('imputer', CustomImputer()),
        ('preprocessor', preprocessor_deploy),
        ('classifier', LGBMClassifier(**lgbm_params))
    ])

    # 3. Evaluasi menggunakan Cross-Validation
    score = cross_val_score(
        model,
        X_train,
        y_train,
        cv=3,
        scoring=SCORING_METRIC
    )

    return score.mean()     # Optuna akan meminimalkan/memaksimalkan rata-rata skor

In [25]:
# 5. EKSEKUSI STUDI OPTUNA

# Sampler TPE (Tree-structured Parzen Estimator) adalah default Optuna yang efisien
sampler = TPESampler(seed=RANDOM_STATE)
study = optuna.create_study(direction="maximize", sampler=sampler)

print(f"Memulai Optuna dengan {N_TRIALS} percobaan")
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)


# 6. HASIL AKHIR

print("\n HASIL TUNING LIGHTGBM DENGAN OPTUNA")
print(f"AUC Score Terbaik (CV): {study.best_value:.4f}")
print(f"Kombinasi Parameter Terbaik:\n{study.best_params}")

# Ambil Pipeline Model Final (Dilatih menggunakan seluruh X_train)
best_params = study.best_params
best_params['random_state'] = RANDOM_STATE
best_params['class_weight'] = 'balanced'
best_params['n_jobs'] = -1
best_params['verbose'] = -1

final_lgbm_pipeline = Pipeline(steps=[
    ('imputer', CustomImputer()),
    ('preprocessor', preprocessor_deploy),
    ('classifier', LGBMClassifier(**best_params))
])

# Latih Model Final dengan Seluruh Data Training
final_lgbm_pipeline.fit(X_train, y_train)

# Evaluasi pada Test Set (Validasi Akhir)
y_proba_final = final_lgbm_pipeline.predict_proba(X_test)[:, 1]
final_auc = roc_auc_score(y_test, y_proba_final)

print(f"\nAUC Score Final pada Data Test: {final_auc:.4f}")


[I 2025-12-08 21:13:18,226] A new study created in memory with name: no-name-b862e902-158f-45f4-8f3c-a59d371d854a


Memulai Optuna dengan 50 percobaan


Best trial: 0. Best value: 0.773251:   2%|▏         | 1/50 [00:03<02:35,  3.18s/it]

[I 2025-12-08 21:13:21,409] Trial 0 finished with value: 0.7732512568416866 and parameters: {'n_estimators': 562, 'learning_rate': 0.07969454818643935, 'num_leaves': 79, 'max_depth': 8, 'min_child_samples': 48, 'subsample': 0.662397808134481, 'colsample_bytree': 0.6232334448672797, 'reg_alpha': 2.9154431891537547, 'reg_lambda': 0.2537815508265665}. Best is trial 0 with value: 0.7732512568416866.


Best trial: 1. Best value: 0.790959:   4%|▍         | 2/50 [00:09<03:50,  4.81s/it]

...

[I 2025-12-08 21:15:20,664] Trial 47 finished with value: 0.7923811065967667 and parameters: {'n_estimators': 875, 'learning_rate': 0.004124613891169097, 'num_leaves': 44, 'max_depth': 6, 'min_child_samples': 45, 'subsample': 0.9995336830201922, 'colsample_bytree': 0.863467294054376, 'reg_alpha': 1.6868919208719138, 'reg_lambda': 0.1930737250440037}. Best is trial 32 with value: 0.793604173473625.


Best trial: 32. Best value: 0.793604:  98%|█████████▊| 49/50 [02:04<00:02,  2.95s/it]

[I 2025-12-08 21:15:23,092] Trial 48 finished with value: 0.7872186105731164 and parameters: {'n_estimators': 698, 'learning_rate': 0.05571612526848566, 'num_leaves': 25, 'max_depth': 4, 'min_child_samples': 113, 'subsample': 0.8763802424773005, 'colsample_bytree': 0.9206747679419996, 'reg_alpha': 0.8581072966250312, 'reg_lambda': 1.3984543772264393}. Best is trial 32 with value: 0.793604173473625.


Best trial: 32. Best value: 0.793604: 100%|██████████| 50/50 [02:06<00:00,  2.53s/it]


[I 2025-12-08 21:15:24,819] Trial 49 finished with value: 0.7917507590528704 and parameters: {'n_estimators': 483, 'learning_rate': 0.006360801274153528, 'num_leaves': 84, 'max_depth': 5, 'min_child_samples': 30, 'subsample': 0.6923545999675519, 'colsample_bytree': 0.8952021171061302, 'reg_alpha': 0.273373862889253, 'reg_lambda': 0.570069243932888}. Best is trial 32 with value: 0.793604173473625.

 HASIL TUNING LIGHTGBM DENGAN OPTUNA
AUC Score Terbaik (CV): 0.7936
Kombinasi Parameter Terbaik:
{'n_estimators': 794, 'learning_rate': 0.008881822912096228, 'num_leaves': 29, 'max_depth': 5, 'min_child_samples': 84, 'subsample': 0.8677638295483068, 'colsample_bytree': 0.8766480738100595, 'reg_alpha': 0.43053528985758927, 'reg_lambda': 0.25327579930230437}

AUC Score Final pada Data Test: 0.8121


In [15]:
import joblib
joblib.dump(final_lgbm_pipeline, 'model_lead_scoring_final_deployment.joblib')

['model_lead_scoring_final_deployment.joblib']

In [14]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [26]:
joblib.dump(final_lgbm_pipeline, 'model/model_lead_scoring_final_deployment.joblib')

['model/model_lead_scoring_final_deployment.joblib']

In [27]:
joblib.dump(final_lgbm_pipeline, "app/model.pkl")

['app/model.pkl']